# OpenFLIM Ops

## Dependencies

The OpenFLIM ops lives in `net.imagej.slim-curve-ops`. This dependency has to be present in order to use the ops. You can either import the package from your maven local or the ImageJ central repository.

In [1]:
// uncomment to import from local repo
// %classpath config resolver mvnLocal
// %classpath add mvn net.imagej slim-curve-ops 0.1.0-SNAPSHOT

// import from ImageJ central repo
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
// uncomment to import from ImageJ central repo
%classpath add mvn net.imagej slim-curve-ops 0.1.0-SNAPSHOT
%classpath add mvn net.imagej imagej 2.0.0-rc-71

import net.imagej.ImageJ

ij = new ImageJ()
op = ij.op()
nb = ij.notebook()

Added new repo: imagej.public


net.imagej.notebook.DefaultNotebookService [priority = 0.0]

In [2]:
// run this if dependency messed up
// %classpath reset

null

## Utility Code

Here is some utility code that helps display the multi-layer fitted images, no attention needed.

In [3]:
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.real.FloatType
import net.imagej.display.ColorTables
import net.imglib2.converter.Converters
import net.imglib2.converter.RealLUTConverter

class FancyDisplay {
    
    public lifetimeAxis, channelAxis, op, nb
    
    public FancyDisplay(ij, lifetimeAxis=2, channelAxis=3) {
        this.lifetimeAxis = lifetimeAxis
        this.channelAxis = channelAxis
        this.op = ij.op()
        this.nb = ij.notebook()
    }
    
    public colorDisp(aImg, tImg, aMin, aMax, tMin, tMax) {
        def aDiff = aMax.copy()
        aDiff.sub(aMin)

        def lutConverter = new RealLUTConverter(tMin.getRealFloat(), tMax.getRealFloat(), ColorTables.FIRE)
        def coloredT = Converters.convert((net.imglib2.RandomAccessibleInterval) tImg, lutConverter, new ARGBType())

        def imgCpy = op.create().img((net.imglib2.RandomAccessibleInterval)coloredT, new ARGBType())
        imgCpy = op.math().add(imgCpy, (net.imglib2.RandomAccessibleInterval)coloredT)

        def cursor = imgCpy.localizingCursor()
        def ra = aImg.randomAccess()
        while (cursor.hasNext()) {
            def color = cursor.next()
            ra.setPosition(cursor)
            def brightness = ra.get().copy()
            brightness.sub(aMin)
            brightness.div(aDiff)
            color.mul(brightness.getRealFloat())
        }
        return imgCpy
    }
    
    public tableDisp(fittedImg, zMin=null, zMax=null, aMin=null, aMax=null, tMin=null, tMax=null) {
        def sampleZ = op.transform().hyperSliceView(fittedImg, lifetimeAxis, 0)
        def sampleA = []
        def sampleT = []
        for (int comp in 0..((fittedImg.dimension(lifetimeAxis) - 1) / 2 - 1)) {
            sampleA.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 1))
            sampleT.push(op.transform().hyperSliceView(fittedImg, lifetimeAxis, comp * 2 + 2))
        }

        println("Z min = " + op.stats().min(sampleZ))
        println("Z max = " + op.stats().max(sampleZ))
        for (int i in 0..sampleA.size() - 1) {
            println("A" + (i + 1) + " min = " + op.stats().min(sampleA[i]))
            println("A" + (i + 1) + " max = " + op.stats().max(sampleA[i]))
            println("Tau" + (i + 1) + " min = " + op.stats().min(sampleT[i]))
            println("Tau" + (i + 1) + " max = " + op.stats().max(sampleT[i]))
        }
        
        // default values from img
        zMin = zMin == null ? op.stats().min(sampleZ) : new FloatType(zMin)
        zMax = zMax == null ? op.stats().max(sampleZ) : new FloatType(zMax)
        aMin = aMin == null ? op.stats().min(sampleA[0]) : new FloatType(aMin)
        aMax = aMax == null ? op.stats().max(sampleA[0]) : new FloatType(aMax)
        tMin = tMin == null ? op.stats().min(sampleT[0]) : new FloatType(tMin)
        tMax = tMax == null ? op.stats().max(sampleT[0]) : new FloatType(tMax)
        
        def labeled = [:]
        labeled["Z"] = nb.display(sampleZ, zMin.getRealFloat(), zMax.getRealFloat())
        
        for (int i in 0..sampleA.size() - 1) {
//             aMin = new FloatType(Math.min(op.stats().min(sampleA[i]).getRealFloat(), aMin.getRealFloat()));
//             aMax = new FloatType(Math.max(op.stats().max(sampleA[i]).getRealFloat(), aMax.getRealFloat()));
//             tMin = new FloatType(Math.min(op.stats().min(sampleT[i]).getRealFloat(), tMin.getRealFloat()));
//             tMax = new FloatType(Math.max(op.stats().max(sampleT[i]).getRealFloat(), tMax.getRealFloat()));
            labeled["A"   + (i + 1)] = nb.display(sampleA[i], aMin.getRealFloat(), aMax.getRealFloat())
            labeled["Tau" + (i + 1)] = nb.display(sampleT[i], tMin.getRealFloat(), tMax.getRealFloat())
            labeled["Pseudocolor" + (i + 1)] = this.colorDisp(sampleA[i], sampleT[i], aMin, aMax, tMin, tMax)
        }
        return [labeled]
    }
}

fcd = new FancyDisplay(ij)

FancyDisplay@32730099

## Loading Dataset

Here we use the [scifio](https://imagej.net/SCIFIO) [bio-formats](https://imagej.net/Bio-Formats) plugin to load time-resolved transient data from `input.sdt`.

In [4]:
sdt = ij.scifio().datasetIO().open("../input.sdt")

[INFO] Reading SDT header


The acquired dataset is actually a 4-dimensional image as we will be shown bellow. It appears purely dark because the notebook by default displays the first layer it sees.<br>
We now use the following snippet to "chop up" the dataset for demonstration. We also display the metadata for reference.

In [5]:
import io.scif.lifesci.SDTFormat

sdtReader = new SDTFormat.Reader()
sdtReader.setContext(ij.getContext())
sdtReader.setSource("../input.sdt")
sdtMetadata = sdtReader.getMetadata()

// display the axis type of each dimension
for (d = 0; d < sdt.numDimensions(); d++) {
    printf("Dim #%d: size: %3d, type: %s\n", d, sdt.dimension(d), sdt.axis(d).type())
}

timeBase = sdtMetadata.getTimeBase()
timeBins = sdtMetadata.getTimeBins()

printf("Time base: %6f, number of bins: %d\n", timeBase, timeBins)

cStart = 6
cEnd = 15
tStart = 5
tEnd = 16

table = []
for (c in (cStart..cEnd)) {
    row = table[c - cStart] = [:]
    row.put("Channel", c)
    cFixed = op.transform().hyperSliceView(sdt, 3, c)
    for (t in (tStart..tEnd)) {
        sample = op.transform().hyperSliceView(cFixed, 2, t)
        row.put(String.format("%.1f ns", t * timeBase), sample)
    }
}
ij.notebook().display(table)

[INFO] Reading SDT header
Dim #0: size: 128, type: X
Dim #1: size: 128, type: Y
Dim #2: size:  64, type: Lifetime
Dim #3: size:  16, type: Spectra
Time base: 12.500000, number of bins: 64


Channel 62.5 ns 75.0 ns 87.5 ns 100.0 ns 112.5 ns 125.0 ns 137.5 ns 150.0 ns 162.5 ns 175.0 ns 187.5 ns 200.0 ns 6 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAGDElEQVR42u1dW24jMQzLJXvJXrKLfCzQDpKMJZF62CxgoGhSjy3LelKax+Px+EGOr6+vH/ScGsTxPLAph5a51oMYeZ/NMvZxnXPlGZMu1GPHQ9MwSgCNfkyfeDF0IIdLof6E6XZAmzGMCHi4ZNKBRIjuoV0zeuvQD5d+Z/nxGg4G+E3EaoLqQAmBoOcv39/fW4tKSZcCFaDbKjfwKCb5v5dhe9r3cDvZLqx9APY136/WCNFQ4r/bnpLpohszUZoB59ZhyQsYfJjez8SIQyWAbjaJAWSQvf/+5rTRzdpFCjnXdQahMteiSOBhtxX1jCLGkZEWXXfX4M/inNKth0u0fQ9TDFXEAJMIPyRcy3xGD6Jn5dIt8w+r8TvHCxgKvOgdCFLg5diYQY8CiCh6Z1fgyXWdv8G7oD30IErFs7OZToUh8sFLU9hv5j4LoLn67M4qBcxAMgIVCDr8cCYljODjlNtVbWRuJQHYmSuJ/NSg2VliX8zVxAZQOHdQJLAaLKFijoMDQaotlBdAZZ6oCkN9P3oOgf8/DgatMVUFdC/cGNpI+jxIlld8E5oziAGkIsr3t+chdDz8Co9mQYXOujUZdkDHsDRxHTkPnWIgdUH5JKKX1+DaTL91cjiZie5JAq/MRP94JMqVYe4+76wGS3sEMTj7Fdp1ij3CamubVHSaezOtn99Jk+k1iBU9mk0q4JW4yTYOO7p/78SwRf29+m4BbXnW8OqmGP6x1aCNhJyZnsN/CYGkoVkCZIstj2XNYjbU7WwcmNqn00bEM6hOcllv8KdLuDLfbXHo6gRWl49l2X7adDSo8k5FrO6XiQXw1gqa28S9ujkrFjo6JmBRD8ggzcqhRw46KplCgJCrOOnefnV6Bu9VxW+2O7jcHwANmXr+sDe1ajnvAH71YBX+MOFE0OWdXvV6LoW4vDQ6v5hX5VVXhnoyCysOgrQhQB6NbQFWC9jjpnmSHZ/WxQwhV+UtPqk7I/3qIc3erGJlt7GKjCkpPcwXrRa/na1HPcbSnQtcDROzznGxh2oXk1U0EQlSWQMs0Wd5pZhTQsRELDLQ49mwxw1ajZ4hgj8MqYVYwy8a9LPqPUERVIx/ZZ7VFzl6rP9XeyfbMxxOzUzsWNLQd15G9C2dFalvz9pgnUKrsmuZANZV3bz691W1xTI2IaDQqR3B363bonYyvICySGCBDoISigWl7pJTiKihN0zOE7+ofHlGkMfLOOgDQ6hU415wm0LEvLNDuQxQiTc8jUQTL89fEcO2RAGRxPMYb6veTsWLKKDVwUxAQsQ9ZLlBHWv3ImBYGiCkS1GlV02h1hWpj7Cmcb0ILRczeNEzzCJJb0p4UlVQJDUekaJuQEhGRi/rwC1I2ohhyKipsEidxUsUd7+m1+xZRC4qOGQtG7urbA48oxafliEtmBY6EjqGCnEbJZOaOFWBOSq6rodAodGFeAjwTmcixHSWocnyHlaSVcs2gEfHeIyPuwNAFX8y3EBWFvHOOLwD5gQ9MH6mqqr2gGGceoh99TpYTE8BhKCzg+iEDjrhxIpoAo02aE6lJBJ4V948zRBFFX6wso2psHBr2DIKykR8f7XwwtoBJaqCkhJMmHBuRT/8qM+8cqjVr5tnYC///M8dIrUbjAsdNOmaUVwJFsFrAyuJMTEtvILkASduzBIsDRbOioMjdT8qrYoCiWTmUKDvDUTVALD752R5NIzLwEALw9xARu+dDsCTCCN/SjF7n59kk/HLq6pu9XKbFFJ105CEWN8NdUsVM56FiFIGLy6/T48VxTI5RcysGCbRBoNYYdgHneyMbswJbOJ9JqgjE8xZ3YntxtbJJ152G5jKfj3tDUVke9MdpMiBnVDrkhrdXMsuc7RxA7u/zUM9jQdJAA2sLVP61rBT/fFN1UPfti3TQs9DmUe3fbIqANBCBzptf+BU8ezDy3DdJrwHIPCs2QmSStujkm5QL+Bkt2+XDCTMCOxCDPY6dnsJpOIA8mDOYYDs9qrHhYJPuGliFmNpWHUxh0T/IRJA4r0pA2RY5Yr+SQKU1glIAsgvFgMosXMctlG3QBJA49jxDy/+1tdDW01/AAAAAElFTkSuQmCC" /> <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAMIElEQVR42t1dAXLkRg6bp+3T9mn7s0s5dU4UbTcJgCBb2q1yxbHHM60WmwRBkPp8Pp//RV8/fvxYfh/9DPkd+/X9Xs73rK5l+m+bvmoLzV739XvkNezvpjdy8vOGrw2/ecjC2MWfPBGIcbIe8Q0GcXs//yZ1nOD765Eboax72r1Xb67BOJ7nTrs3JTKmqlF9/f7nz5/vwRSdN19xlxWw5wCk6n6g1/ptHIgRjhxCddO60f8Jdzq13odlAlysVy33AbGOxhEVb9Ad5mxfzgufjHWda+w6zU/gMhafPRtfq1nFG8mfyr4OkG8xWHFe/FS4eCL+uJ7+Cp/SgL3eQ5MimzgZ0qITexrzEH8/i2ArLu0tocMNADuu+/Ke/SzeU2/QFFLv2EfjwZjL8btAoNP1doaUJ4FgyAOccpunPY8DrDoP0dFikLK4jgziREbi9irV92vycu8QNlTSJ8SjnCRpJopBAZFWY8OUMu1pV7gzksx4uskgF81OrnPeujPQdsq7vEHM0pC5zC5UuYDmPLjtRjDpJrKeSGdQuB5/4aWT8nXfuEHK9alElT9te0NdXgV9LKePnmbU8zVUVLGTMBmrvzZnCACdyLuPFdmWf/c27X5H3cIV4jqYPifou3vt/3iAqVTq5U0UkueZSIcLQtc+UuQEkHo7QHP2aID78rwT43TlVbDniOkVmXq1RgJcf69MGrmYExWyk2BxQlgLv7YTBLr5g84eQ4eXmRCWNmQWHCu1a2xw9g+6qmZ/Gr5okqnPU8HK76pudfe+bBsXW13samUzhg08bqtpoLqBT2gmOY3o2f0UmMq5+n2VRlX/phovM4FLRwzvotq3jSFdZEVXPb0CNO9/o4SBk70GEeMohAcf2fAnEzRs99NpiRpxoM9w+0+lh7Ms52mHQfXcNlWwA6i5R8+c9lwoI6mg+xWOKgLm14wysY6uURoynQWfqPYyXOLm0xR2gc6U56lEiyLjUllMc1HofVM31DVd4zt7Yh03Uslw1AyFMCivS1UsuUOR6wwRE+GmU0iS4A9f/p1Z61vmA2RrQzMFlfKekN+ljSGn+gW6O3tdVcHJUnMrSOzg+jsrhE+r9VdPq1qMMjawPkPqtdoIxDhXCuLIeDuN0TlHifUOSPgNNYFq/MqQKkOKVLyLa5iSo9DirIO0h2NHfX4it0eFKm6dgqIAUnN6p7qKWN8sU4eeWNUVTqLpJ08LJw5uLyEzBaaYz+gcYOFQ9FR0iiVN4Imp1Z3ziE5I2DuLZU3e62O1sic92qUK5J50De5eBKg3UMk30T53FCmvcIFbwePaA3eL2O7/VzUNBYQuFUHV04Bsphrj1FiqeLPIGNWMg0mX7/v0fWiqZelF6j6ny6+2oO9AnOuE7sgoJjtxA2ZmbhHqQW+G5NvcHVMXWJ/kJSqDl5EKXMZK7srK6M1X5h+g7j661m0amFk4uskr8mJF26oPVOiQjyHXdD0c0V5lniEzrF+/fkGYp5Im3j/jg5xABJygzNVOlh2dCjcQVQcyrd7/+3XX16+8qPr8JKKur4FNpcS4M5LO5sydIV4BEmuMapEo+u/dY2QhMRN6Xr++D4r5EOAg7G7lUVGI

Shown above are images from channel 6 through 15, time bin 5 through 16. For the rest of the demo, we choose channel 12 and perform the fit from time bin 9 to 20.

## Hyperparameter Setup

Prior to fitting, we set up some fitting parameters specifying how the fitting is done. All the settings are described below. The commented settings are optional and are set to default values.

In [6]:
import net.imagej.slim.FitParams
// import slim.FitFunc
// import slim.NoiseType
// import slim.RestrainType

// create a new fitting parameter set
param = new FitParams()
// the dataset (3D image with coordinates (x, y, t)) we choose channel 12 in this case
param.transMap = op.transform().hyperSliceView(sdt, 3, 12);
// // the iterative fitting routine will stop when chi-squared improvement is less than param.chisq_delta
// param.chisq_delta = 0.0001f
// // the confidence interval when calculating the error axes (95% here)
// param.chisq_percent = 95
// // the routine will also stop when chi-squared < param.chisq_target
// param.chisq_target = 1
// when does the decay start and end?
param.fitStart = 9
param.fitEnd = 20
// // the deacy model to use, in this case y(t) = Z + A * e^(-t / TAU)
// param.fitFunc = FitFunc.GCI_MULTIEXP_TAU
// // assume the data noise follows a Poisson distribution
// param.noise = NoiseType.NOISE_GAUSSIAN_FIT
// // the standard deviation at each data point in y
// // NB: if NoiseType.NOISE_GIVEN is used, param.sig should be passed in
// param.sig = null
// // initial Z, A_i and TAU_i (i = 1, 2, ...)
// param.param = [ 0, 0, 0, ... ]
// all three parameters above will be fitted
param.paramFree = [ true, true, true ]
// // use the default restrain type
// param.restrain = RestrainType.ECF_RESTRAIN_DEFAULT
// the time difference between two consecutive bins (ns)
param.xInc = timeBase
// // generates the image of return code
// param.getReturnCodeMap = false
// // generates the image of parameters
// param.getParamMap = true
// // generates the image of fitted data
// param.getFittedMap = false
// // generates the image of residuals
// param.getResidualsMap = false
// // generates the image of chi-squared
// param.getChisqMap = false

// the index of the lifetime axis (from metadata)
lifetimeAxis = 2

2

All of the fitting ops takes the same parameter, the fitting parameter (`params`) and the Lifetime axis index (`lifetimeAxis`). The rigion of interest (`roi`) is optional (see below).

In [7]:
op.help("slim.fitMLA")

Available operations:
	(FitResults out?) =
	net.imagej.slim.DefaultFitII$MLAFitII(
		FitResults out?,
		IterableInterval in,
		FitParams params)
	(FitResults out) =
	net.imagej.slim.DefaultFitRAI$MLASingleFitRAI(
		FitParams in,
		int lifetimeAxis,
		RealMask roi?,
		RandomAccessibleInterval kernel?)

## Performing Image Fitting

Once everything is set up, the fitting routine can be easily started. The op will generate an `FitResults` object with all the per-pixel results assembled into images. Specifically, `resutls.paramMap` will be the image of fitted parameters if `param.getParamMap` is set to `true` (which is by default), and `resutls.fittedMap`, `resutls.residualMap`, `resutls.chisqMap` will be those of fitted data ($\tilde{y}$), residuals ($y-\tilde{y}$) and $\chi^2$ respectively if the corresponding `getXxMap` option is turned on.<br>

This images in `results` will be of the same size as the input dataset in X and Y directions. The result attributes (fitted parameters, $\chi^2$, etc.) for that (x, y) coordinate will be layed along the Lifetime axis. E.g. `results.paramMap(x, y, 0)` will be the *Z* (constant term) for the transient at coordinate (x, y), while `results.fittedMap(x, y, 4)` will be the fitted data of the 4th time bin ($\tilde{y}_4$) of the same pixel.

Here we demonstrate the most used ops:

### Initial Parameter Estimation (RLD)

In [8]:
// spin!
rldRslt = op.run("slim.fitRLD", param, lifetimeAxis)

net.imagej.slim.FitResults@2251285b

In [9]:
// The fit is pretty noisy
// Here we display the clamped values (z: [0, zMax], a: [0, 1500], tau: [7, 14.5]) and the pseudo-colored image of channel 12
nb.display(fcd.tableDisp(rldRslt.paramMap, 0, null, 0, 1500, 7, 14.5))

Z min = -514.6663818359375
Z max = 23.764686584472656
A1 min = 0.0
A1 max = 2628.95068359375
Tau1 min = 0.0
Tau1 max = 130.3522186279297


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAk6ElEQVR42u3dR49dWRHAcZNzzjkOOec8wAx5Zsg555xzBgkhQAQRhBAIBELAAhbsYMOGPUJ8D76D0e9K/6fydbvtsRn3c8+1dOR+N5xQuerUqXvixIkTJ7d2vW4bEDYC2NpGAFvbCGBrGwFsbSOArW0EsLWNALa2EcDWNgLY2kYAW9sIYGsbARxtu9nNbnbWdq593eUudzmlbQRwPVjkDW94ww3R1wcCIAnucIc7XCuJsLVjvLhXvepVp/y+zW1uc+jzd7zjHTcCuBTbPe95z+X/W9ziFidvectbnrz5zW++/D3vabe//e13129729vurj/sYQ87pb9b3/rWS/P3gx/84I0AzrdBxmxnE9/9DYFE+UHP3fnOdz7t2q1udasD+4LEO93pTguyEcLtbne75d6Nb3zjk0996lNPeefe9773KQRAWhykShqre35fyu3EOT94kSgSInDpety4GwFA5k1vetPT3n3EIx6xILfnb3SjGy1/v/CFL9y9f/e73/3kZZddtnvnMY95zMl73etey7hxfeP67XpG5JQaCGRKl00CXECDlCREQAZ4XAY5Abx2v/vdb/du97WHP/zhOwQS3c9//vNPfvCDHzz5mte8Zvl9xRVXLBLhPve5zynEdbe73W0hLHNARK7d4x73WK557mlPe9pOnbgeodzkJjc55f+NAM6jPehBDzpNfMdx/l+L5sc+9rHL/5DW9U9+8pM7A49vD2H+j0Pf8573LP8/5CEPWRCI4CAWMUCe/yMIBEBy1J/xqaQnP/nJO2KJ2G5wgxss7371q189q4G5EcAqGDMJAKIghV7G0RD8ohe9aMfRxDpEaJCTq5eIT4pMVeD+Xe961x3Xuo4wIFD/xqUa3PebSoFEc/O/d+J8czK3pNRDH/rQRaUYgwGJeNY2xbEigHOJxJ1LlC7RqeEciMKVfkMqoAOs5xLF3g+5EBOnhpxnPOMZO6Mt+2BtU+grru/+2pg0D8RhfFIB0jXvhGRz8dx973vfHVF5hkrxjOuXoCS4eINBFK4FPEgBLAB9+tOfvjPc4nqImsYeces9rhygX3311QsSceNEuj6TIl1LgkCicee9PAV9+J994X3IZI+89rWvPfm4xz1uN6+pBt72trft3E4ErC9rmMbiRgADCXEMDpsiGyKf97zn7USsZyDKb8EZyPN3faSDX/3qVy/vdT0jMYmwdt3800+xAM/5HTcT40keBGGeL3jBC5a+nvCEJ+wIF8If+MAHLv34TRqZw1RtnnGv/jYJMPQ/NwywAdD/kPnc5z534Wp6m78O+HEa7gLkrP9UhfeI3wjnxS9+8dIOcg+nbRBxXH755TsJA9HmkE2gb38/61nPWv5OJRgriUOaTTskQp1xgmnEXi8JAEAgEEcUjXON6Pb7yiuvPPn+979/x6mQDqm5ZlQExGcbJF4Ri/8RzGGxf5z6zGc+87SoHoJCSNkgSZ7pDWjeSaWYE12PgKcEW8dJEEVxhYhtqp1jTwAhCXAtHJLiXKKyoEwWNVHumvsARuwjmHQqafCSl7zklOBQlrt39UFEn0u8//73v/8yn5Cdm5eVH0Hg+inCP/WpTy3PQ7w5eSd1Fpe3phlU6u8zRTaPrQSYwRF/4zhADUHcsN/+9reLgfXIRz5yEb84K33sWW4alcEAdI+R9oAHPGBnMEIAcfyud71rx4lEdmLd/TmmRrqsvYSQGLKNw+/PQOx9xPGkJz1peXYaelTW3GeYYj/VMQNWx54AcAlxSTRDCq7rOu7GsRBDtPtfm7pV5M7frgHct7/97V1swDvZBgG199rUMR7CWu8xdD9ETfGN8AoV68+c9JP1XzBpegMIAiGYm35e+cpX7qSW34iah4P4ERlCKc5wLAmgYI0FAyIdDlm4iDvlGp0MKYyuRz/60bt3ps/e30S1BjmQN6N7AE0aeB5RhdT8+0R0HoLfIW+tj/P/uX55LES2sdyLYGdcxDN5ELme5uG5YEBquedZ95Ma3stjOTYEMMVekbR5nVjOsua6EaVx+jTmAAeypyqgAnAboDLKALZAUMblHHsN2FzDabWvn9EnDjWuMQsamZe55j0gQsjMQHXNXDyfXWEMz0W8JBfJQqVlDx1bFYDbM3wAEOACKHHoGoC6ds0115xGQBlZEEJCJMJ/9atf7SJt+opb46qCSQhixgua04xorl1Ec4KcKR1cN0fziRibn5bPH7Eb/ylPecrSD+mmTx6O/0mqiBrx63uP3MP/X2fT0MEhcQ2EAEDx9sQpBB8ULSR+PYPD7ODRxSTD3Od3v+1aSJ/ewUQ63YvrcOUkjA984AM7+8GcSJl+Z/0fJKJTR8bnOUSQCOBNb3rT0o9r5k19GJfLmqRC3CSCOe1JkOjCOymkC3isYYimNy0at0A06xsiPQNYEAoI7f9PvRgy3/72ty9cxW7Qn8BNYWHvaCF6EiHAhzySxHPFHULy3NKF0FxBfRufZFmniHmukLJxzMsa2tG0VsanOeuruIH1+htczNc1zzAajwUBsMxReiHRdLdF8p/db1cuPYz6AbKtX8ABfIQA+O366dO9dgq947nHP/7xOwkys3MOml86uudCWN7EdOHWu4u5iSKMeQn6sR5ENTOEEJ1+ELr5RSjGAwNMQgU0vuclpJSCdkTBogvj/IAEiABi4a4HLNTfdYB84hOfuACFweV/gIOIjCNAoztxe+4kYsq69/4Ux3MuxQH8PS3u9PX0x+nl5zznOcu9SRzTqDRH70+7IMIwX+9BasZt4p2hyg5461vfung/2RLWxf7JZsjjOOLMovN7cW6nAjixRmQSa/4GOMiCQBwIAIALOAih4M0Pf/jDhdOJToQEGPxnBOMdUgSQIax4wXTJUgHT84CIuNvzUyVEZJBpHmfK5nGfoVn8v76yESLi4hiNnbvrN8J/4xvfuFwDA0zimXe/+90LAVj3HuQRXNiBi1y3kjogDHBwIqS94hWvWDioffPpGUjXeuc737kABvF4n0j0jMyff/zjHwvgEEMICHEZb7XsgRCt/5e+9KU7ZCemSYbcQs+XsDrVRyJ6buwINbPwyx9M8l111VW73+Y04wMMQaKfrQDR1m8tz372sxfiSQ2ayxGmpJ//yxl5cRvx5jfCgEgA/vjHP77z93E/QJIMRGjiFHDoc5JDH/oFJO+E/JCciC5QlDQA8CnKSagQk+8/t37NI1XFpph7A8adHo3ncm3XKWrZDhCIGKwF4qmC9h1KU4s4jQlGc+/gYibeXjABtKdObONwi8ZZ+cG4v7CrxEwcBcEvf/nLF6scgBEFzodwcXdi3zPu0f/6cbAjFw1C2oUDLAhO/NPlgIjT1vbJ2itAQB/72Md2MQS/iePuE8tTLbhvrtMF9bc14+65gZTYRwDmWQra61//+oUIEB11kJfhWkQ9Q+F7TQBxTbtvXaPrCvDgesgBCMABlNwwHB2XZB1rOILKYChBPgBCKE4qB7ANoJm0kb423rSizRFRHLQdC/jTfWxrOMLWpnQwnwzRcgPy4xtDHxgAkRa9pOvZEdaEWay5sLHx51a0vme6+t5LgGLrkICicTEC8Lf/+fBEoO3atkJJAAulHt7whjcsnEVCQM607gGyzRxAdg/ACtYknkmKkIB4IBbhkCzGmXo1UZ0LCmHt1yPO8gCnC2st03jEsamIud1bzF/j5pkLTodgBG+djGP5ixFv4eFsDP1Mr2ZvCQCQIYKxA4i5W4CG4t/xjncsFJ9kSAy6hyhwNWAgDM8A2BTdSQwSAbDcS21AUFJAS+9DHvsiHZ6HAtkQZb5rHzvC7ewBLs+IRECFr2eo+vOf//wyp/e9732Lympnj8uXUfze9753+e198Qv9IuDp+oHH3E62xrn/sdcEkAvESsfRLYgEIOK03B2

### Refinement (Levenberg-Marquardt Algorithm)

While the RLD fitting op can be used to quickly but roughly estimate the model parameters, another MLA fit can be used with it in conjunction to give more accurate results (lower chi-squared). To do that, either `param.param` should be set to an well approximated initial values, or `param.paramMap` should be set to a per-pixel estimation provided by an RLD fit with the same fitting parameters. MLA fit can easily fail if the initial values are way off the final results.

In [10]:
// param.paramMap can be used to pass pixel-specific estimated parameter (Z, A, TAU) values to
// the MLA fitting routine, which will fail on inaccurate initial values
param.paramMap = rldRslt.paramMap
mlaRslt = op.run("slim.fitMLA", param, lifetimeAxis)

// z: [0, 24], a: [0, 1000], tau: [7, 14.5]
nb.display(fcd.tableDisp(mlaRslt.paramMap, 0, 24, 0, 1000, 7, 14.5))

Z min = -2745297.75
Z max = 4613.15234375
A1 min = -4608.0615234375
A1 max = 2745299.0
Tau1 min = -8.7281201138672599E18
Tau1 max = 2.865592559164457E18


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA2JElEQVR42t3d1Y40yfHw4fXfzMzMzMzMzMzMTDJbMslwYsknhquwfOQzX4evZT895f3N5ptvVXf1zLy79jdSq6erEoMyMiIy8rrrrrvu+pvyc8c73vGK3/583+pWtzp7drvb3e6q97e4xS3Ont3+9re/oo3x3a1vfevVfsc2D33ufOc7HxzzXe9619Uy41jHT+PxN87x2OdQW4fK7PlM7RxvsOf3uMc9Tu5snvT8u8F43v/1A9h7ETi/v8Md7rALuXvG7HP3u999DzA3CXBPmZ77Noa1OeztY/fHxO53v/tdhXydQMBezjmE3DUEndLuGvIC0lrZ2972tss7cxsJbKvt29zmNlc9G6XMSPiNu3ZHIt1C5rH+t+qOMJrbu7TPi1/84lWxegp3HJIi2lfHZOLy/v+///u/qwhhC6B7OOe8HLI2JwSHkHDh/H6PtDlP/8HumGgfx3MKYW189g2wQY0UDUB3uctdNolgbSLjO4BEeFvl4vpxkvo8tKZuAURbCG7s69K56QAjHEPUeZbXS/k0sEMS4NDgQ8gxAlprn+g9hIRb3vKWV3Hdmk4xPpvbG7nTWA9JmDUdYe1ZbYbkWSI0hrW6a/O9loS4WwmcOWoLUDMi733vex/sbJQQhwhp7R1ARwR7dAZlKxfwRyIadwvjuj+LVHUugpRj3K7tsf25r0NMdQ10gdOpcM/2YwsI1Y0wKjcqdTNCIBVBjsg8dRs0E7S/+973vlc9G4nkPMg9b9m1LfHeXct5iOHRj370jQRwSKPe6nBv+QYH4cfqNOG4eE2/CGlp+iMR3OlOd7riu3IkSZIr7vZ73NOvKXtbxHZsHm0Z523tFrKOSbdLUPS2P2uKVoDq2aG1GJIOravHJru2TfL/TAyjCH/1q1+9bF23uGDvjmZE7DHRurbFHaXXrI801nEulTWnxDzYKlv5m1ofuG40wKx97n//+28OLECviWKTbuL+jk1MW5UBjNbicU3EWZ4pS6MfFTLWOkRxz3vecxVpswVy5vpZC/ebmDy0zMwS6H/0s/0yG8GeffgMiJDvHaTMis+plsM1QB/bOs0cekw6ILRj1tCRu4/NaW13cshYdqoEuCRpceUkRwBtUfgolg8ZJfauXWO5pz3taZe9zdnUJy5DkZsVyT07nfMYqiK6U5a33QQwis01zmqNb7nYGsSp5t21z7geJt4bj36PbdFmgg1BJJKtn2/tWE763fvaHts/tiu46Nas8Uya+VG7yGUwxRkBbBlAMtfuMfbMeoNJWXdp5NoZNfND9oVTzZxt3SKWsc7DHvaws/H3fNYF7nOf+6wCejb2NMb+5rG1fKxJzLayewnqItvvCy0Bxyx+s9VtHtyaNjy3HWB979lOQkYK34iI0Ts3vict1MlANQJ81rYzZ/vMiIlQRpP0GpNsIX1eSte4eA1Ox5bcy9IXzn5vWaTmNXPe0/pe86JtSRRl11yqW9JlRGrLzrg0jX14PnoA42ziXz3/A+IaIL33udvd7ra0ea973WsZ6ziuxoIovJsNVqOuNBNSv9fE9xbxX8ut4FVtH9NK5+fnFWEBaUsbnn0S+skWAQEQ4w+CvVPuQQ960PKdDgM5EPmABzzgbMlJxEM+BCOKUewbz4Mf/OAzolcvxxEisC4nrSIgZUYCHUX8uL2cic6z1vlxlzQTR32Z25oj7BQ/zA5iuNpyNhp7QnhlTM7gKzNyy7xvX3OTzojHVSHd/w95yEMWJEOk3wAdsI0BIQQcz5SL6z3TJ8LQT8aiRzziEYs+8MhHPnKpD+FPfOITl7I+2rdkeIeYEILvdIjm1tiMQ/m8oaMkAK9//OMfyxj9Dz7HNPeRqYJd8KWAew9ZW8y39nwPo95Q5j+AG0X+1ppzyGh0bC1fWy7SzAEz4oEEY4E8zx/4wAcuv5Ul6o3Nc0gALOOJWCBaOe8f/vCHX/+EJzxhef/0pz99IQqfxz/+8Uvbjfdxj3vcIjEgW9vaISGMx1ggHBGMkumhD33o0r+xtUPSr76MMf1hXC5mZjhmgJvhOSL0WKDISfaWrciXeY07NV7tWFxgbULeYx/72AWgkACpIdtvHwCPuz1/7nOfe7ZLicsB+wUveMEiYiEGol/ykpdc/5znPOf6N7/5zde/5jWvuf4d73jH9U9+8pOX/vULeRGFPozjUY961ML9LQt2NGwT2gV4xGQsxquO8bQExb2z1NzDmUmakRGziO4xMJ17l3CqgSTNtXpNaFwSxnJrgRHVfdKTnrSsyQAIWDjWe6LW78c85jFLWcjAiZAO6IWx6dszyFSeiMfREAqZL3zhC5d+3vrWty6AgkjcjVAQCWQbA6Q+5SlPWQjC0kCSNB7ltfmiF71oaUv/kKUvddXR7ijFZqkHNltKoDmkG20hNlvGNVEOD0W5nsczOP6/tvYpj5sAFrABGTL7ACSug4QmThR7hxAg9hnPeMbSNtEd5/i/b21D8POe97xFgugPp5IIkKldSH/Zy162tIkAEZ2+mb/9VgfxKYvAkgSkB6liHE996lOXcZFM2mvJMA4EcmhruAbzURcan+v/kAHpwnaAUdT/+9//3txvHwp1SvyNiuMoCcYATe8BGYcDog/EWbdNFiIAGHH6IAaEgpMCckQEGWnNyuHQFDtI05ZyuBVCITqTs+eNTXskgnEoox0f9Z///OcvsEBMLRcIRzmfd77znYtU8cxY9Jt9YUs0t6SM4XGjf8HStNcOsCcA5YAecFgBicP8GfSM3LW4/jWbAJHaXhuQIQtycSHkROmepxfgPpwL+IgEAiBLfSLbeu45xP3sZz9biApSIaC6Ju9/iMSV2vA+JbJtobEgyNe+9rVLuWwIRD+k68fYtGds2kCUuNVYfMAqRdT7keD//Oc/b3J0LuFDxrDRvXzJVsTDIdtjh2uNjLEAI0Gk9cadAIWqAQYAiX8Is05S6gC4ehAad+FURKLus571rDMLn29rPAnw7Gc/+0zyABYEQHCiXd/GmW6R2xiB6CsfA4RCPknkQ2JYbowR4RhLSFcOQyBGz7XvufkhUgRknghC+3OI2ihZ1zhcn3ttLnt1g42dwmHtdKvx2XN4yHERUiAXtwMYAOM44hUgPcdFAP7KV75y+Q3ouFN5yMeNxgNp6ivztre9balvaVC2nUK7CdymH/W870Ncqw+RdAXtf/jDH16QBjn6pEcQ9/ryHEFAsOfpDp6HRH1lG0hx1VdbXc/XkJ1JejT+zG7wfs/1PZ+32LPp/Agu/4N8leKcCs6aa/H8s0KzRSTaMmnrY1s7AAE0AIYs3E/xCpCA+8xnPnNBclznd/aBV7ziFQsw1dMeJELEGoDMidJX234rS6Rr3/PagTxcq883vvGNi3QhlbITkDIRJeRqI0U2+0TKq9/moZ7/W4pGp9ReET3qZyF6zWu4dws44upMBzgk5rfOoyXic4jM6z6igrTWc+UhM+0eINvivOlNb1rKWNMBCWCVa0eAkBCJ52n+gIqzIcIygIOJ9hA56h+jMcYygOshyHsEoR5CwLHCzfSn/4iOeH/Xu9511r9vz7WhfkSnbeNVz5LVDsIn62Te0byl4LQWrXRsSc7IdJF4waXdLUPCmo/gkDYJeGPAw7gu+uAkorY11roOaZCPw1//+tcvywGRC3DKIQrvcKLnAB4BaRsiEMn73ve+5VsfuNT/kApZ2mg344Pw1M8WUT3PItjGqT5JgFA8M25jIQEgoI/fEB+nazNzdkpthqXROBTiE/uzbX9mzBE/e04jbTnrVnUAg6zRUaSOA17zg8+KRQpTFrn24wCKW3EtpOAIyAegtG59AQriMQZItnf/xCc+sSxH6isPEY0N8PSpvOekA8IwH88hJkLxf04h5fMj6F+/xppiZ4kg3o0FMdE/PPdeXfW

Note that the fitted half life values outside the cell region is diverging.

### Global Analysis

In [11]:
globalRslt = op.run("slim.fitGlobal", param, 2)

// z: [0, zMax], a: [aMin, aMax], tau: [aMin, aMax]
nb.display(fcd.tableDisp(globalRslt.paramMap, 0, null, null, null, null, null))

Z min = -16.0
Z max = 18.31560516357422
A1 min = 0.0
A1 max = 1100.790771484375
Tau1 min = 10.749341011047363
Tau1 max = 10.749341011047363


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAA2MElEQVR42u3dh7MsVdUF8GdCUUFUzCBBMWNCRUAUCSooohgQBQOYA0FBQMAABqIZDAQxAUZAMYCigkowgWICFSVqlRZQ/gP91e9Urff169cz0z3Td+59+G5V18zt6T59zl5rr73P6dOnly1btqzqut3udrer+hx/+9vffqV9d7jDHVrLyv+TrlEvs299msc36+f3tjKb1+xy3Wnq1vecgbbhCVA/bpZGjSLJuGPHEWma+ncFZygAh7LdVATo0vAuhpi1oW0gxgsXwvuGBEU9uxB1jgDPrgDTVnocaKM8pi+BRh3fp56z1GMe2wLXZbjKzNvz+gA3tDyvYiBPJsC4CrQlc128sb5/GhXpkvDVJbctZPRJ8sYRalriLyUl6a0AiyGNyu8T7xfK49rI1zX/6FP2EiDL0olr0xpuqGS0j9rNsqUrvETCwny8qk9voq1/7vOOd7xjrz51zutCnGmMParrOWu+MtdtiG7VOHC7At+lC9okQD1ctHlWn57CKNL1JcMQ3j3XruIQzB8Xy/smkEnq2rw9xq3nCF28z/H1EcjUrf45KVGdZnBoFekZzCZ/o7LoLixuO6ZrnG8DYJwX1wHvO4DTJ/dYBbd2TxpyuHbU+P+8ks1R7Zs00jlk3J6lmz1XBehb0XE3VGZVlj5xtR5y2sYG7BNWbKlzQkM9DPQh7xBJ5qw2GiQHaDN6V+O3GaGtLz9t92qcLKdMdbQ12xFwA/yd73zn6k53utNK7RsVSoZIbleJewFdk7o+I3jjpLbLCF8bKKNA89saa6yx/DPgZ8tvSIAA2Z/v9jsHQXza36Yi04K+xEcJhxnQGDXsW/+9ntmPu0EUANP1C3gBvp6955iADLz73Oc+Zd/d7373as011yy/Adf3u971ruXTZp/j7FtrrbXK/9lynvLqpKmTr009hhwPmTWcLkg3sA3oacf562UEYJ8xvv/XWWedAshd7nKXchwwAlBAvPe9711tvPHG1QMf+MBq/fXXrzbYYIPqIQ95SLXeeutV97///ctvG264YfnNd78/9KEPLft8v9e97lVIc7/73a96wAMeUD6dd8973rOQA5F8Tz1cNyoxL29fQLWYbbRqXPxsG5ipG60enxkZ2GuvvXYxer4zvN9CgLvd7W4FAPuzOfZpT3taAfRhD3tY9chHPrJ6xCMeUfZtu+221ZZbblk94xnPqLbffvtql112qXbcccfy/0477VQ94QlPKMRxzqMf/ejqSU96UvmOKAj04Ac/uJBlo402KiRBLL896EEPqu5xj3usEHJWMemf3A2cNrFpO68p/wGf9wENeIDkzeuuu27Zt8kmm5T9jB3gkeO+971v8UZgAc/xQH7Oc55TPeUpT6ke85jHVM9//vOrd77zndXrXve66o1vfGP1ohe9qHrNa15TPt/0pjdVb3vb26qnP/3p1XOf+9xChEc96lHlPL8r45nPfGb12Mc+tnwiCzIhiOO22Wab6nGPe1zZkEHdkj8s9ISVwckza4yZZkQsMg9I3sWw5JmX8UhgMjhjb7bZZoUEgPa74wGPILyfhyqLbPPul73sZQV0IB9wwAHVe97znupd73pXteuuu1b77bdf2QdUBHj5y19eVOFVr3pVtfnmm5dzge13/7/kJS+pXvrSl5Zyn/WsZ5VjkUVZjnne855Xjn/qU59aFKdrWJh1ssrAoaTfxZpDtn1VI/IN9K222qqALd7ych7H8MBn8K233rrse/zjH1+8jWfazwN9FxacF+VAnn333bcA/pGPfKR673vfW+2xxx4FWOcdfPDB1f7771+9+tWvLuDvueeeRRVe8YpXlGu63m677VYIJnQ88YlPLOHizW9+cyHJFltsUa6BGEhAcV784hcXUlAcaqAeXcdShphkOwBhlnUasx81sjZqyLVtv3jJQA9/+MOL/PIgQAIYqIDhcbwLKIzsf98ZmETzxhe+8IWFQMqSvSuXImy66aYFxBe84AWFBKTf+W9/+9vL+QcddFC1ww47FG8FsnKASGkkfz5dK6Fk5513rl772tcuVwtA26d821577VVIhBBCyZOf/OSiSOrVduOqK5jTzEJesBlB48bIu9wwSRnipPjNQLweIBQgGylFCL/55LU2isBrGR5owgEpdg6QeKTETHjwCUSkoA6ARiyEk9g5H6kAluRPL0CypzzX4OmIpB5ARQBh5H3ve19RAmSQJ6gXQtj3+te/viiJeiEgJaBKbd3kpTAo1LjWeAaOY+q426n1UTqyT+bFeZINLN7y7Gc/uxiRTPPWN7zhDcWAfuOpvMz+V77ylcXQjpMTAJLnOR6QwgSvA77EUUyWE9jnt3h9zlU+1QEyYiCBEEFdENP15BHCgZyBggDe9w984AOFFHKMffbZp3rrW99aSENNlOf62uraElzE79NDWARCtMt63+HMUd6v8TxS8gYYoDIWL+TpPJRXMrZNTOWBPFKY8D9gQhLnAtr5pBo44jRAdRv1CHg2Bcg4AmWwIQPAgYMgzlNGkk6evffee5frIWZ6EOpB7g855JACOvARUh0cz/PT+9BOIUn+4HrUQF3awuySGgns+vBDl5sh/uf1+u9iIeOTYcAyrBjPeMA88MADi1EPO+yw6i1veUv17ne/u3gaYwMdSOK443kf72RYn8hh44GUgLq4ti6ZcMPw6S6mW4lYzgX2dtttV2I7QNXF9RDUb3IB3UhlUyoKceSRR5Z6UR4qgLDpLgKfkiA7kgknVMc1kY0adL0rOtSzF1PdDZyUENbvpI1LajSW/DEC4zKSmM4wkj2GtTE0LwOCT6BQhEMPPbT8T2J5n+ye4QEkhPgEZIjEg3Urc7MHAV0b+Ug+ACgHFQKosABA8Rvw6oEMuoq8G7FcD+koAU+nNsgJ/N13370ohd6J+lAduYTcIAmmMl2LGmTsYgkOEk03AWRUFzAje5I98sszgCrJYwyxmwrwbESQ4CGGTx4nBvvt8MMPLwanDIAAClIgCmCFD+XahxxIopsGABKsq8nzMqqYMQOJHa9FEKqkuycMAVSZygc+NbIP2MBMPd7xjncUYiKQnAL4QoprUKGEA+c7DsF8dw0qRBW7zi+cS5I4bopy3wcvIv0MwjsyYgZ4IOmHA4qBGZc38Sy/J5byHkAgCYIcc8wx1emnn17ORQpl8mIeBVDEQiBqQF1IrjjM2yWdCGVjeIQg7zw19db987s6yOQpi2snsfOpzp/85Cero446qiSDVEfegFTILvSoM7C1wznCVsIaQssjhBF1zvBx2/D5rBNyBu8GjpoXUL8rl33iHykU6xEAIIxKQsVO8Z5X+P9LX/pS9fWvf730zRmaVyILEvBkm3OOPfbYMqDD85zP63i5czJOjxCSQPGdcQGfDJ/HIwjvJPnUgyoAT9LIc0M4KoNoQPvgBz9YANUF9L8NYamOewAZzVSeRJI6ONfxzkWUeL9PvyOO9iFdn8fl5jIpdNJj0W0xvz6jRuwFBIAYlHQyPK8BMuPxUgQg9TxK8seDhQlkAQJwebH9jPj5z3++EEEsVk5u6gAUYRANWSiHLSRUD+VIzOQiGc9HDt00hPFdoogk1EbugZzUKt6srhkJ5MEIQ0EMRFEjnu1c4COqUJa8xkbpnM8eH/3oRwtxtVc9J93uXRLPBYyqREa6EvexWrKjwbwVeDwU83kuo0qw/M4w5JxxySMwGZNaRMp5K/IAhAHlA4yLXMkTMlYAWERwfXfseLjul+/AJv9RjKhZfV6AMlwbyOoYzyXjrilkuYbzJZVRNvkC0kpUjz/++OV1/NCHPlRGIvVo7EcQyiBfUa7yDGhRq1GjrHNTgi4jVKMqk5sf2MwwibdA5y3xDDLO48VSRjjiiCNKl8p3+4AvBgMpw7ZkVYj4xCc+UR133HH

Two component:

In [12]:
param.paramMap = null;
// set # of exponential components
param.nComp = 2
// extend free parameter settings
param.paramFree = [ true, true, true, true, true ]
globalRslt = op.run("slim.fitGlobal", param, 2)

// z: [0, zMax], a: [aMin, 1100], tau: [tMin, tMax]
nb.display(fcd.tableDisp(globalRslt.paramMap, 0, null, null, 1100, null, null))

Z min = -26.190486907958984
Z max = 17.251169204711914
A1 min = 0.0
A1 max = 502.6582946777344
Tau1 min = 59.066864013671875
Tau1 max = 59.066864013671875
A2 min = 0.0
A2 max = 1122.9990234375
Tau2 min = 10.158617973327637
Tau2 max = 10.158617973327637


Z A1 Tau1 Pseudocolor1 A2 Tau2 Pseudocolor2 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAly0lEQVR42u2dZ69tVdmGD4IiNjrYAalKUcHeQAHpRbAhAoqI2CtYQA0lUiQKJAIxRPwAIQSiBj/xwR/AH/AP+EP2m2u+Xst7P3vMueaqe59z9klmztprzTnKU+6njGeMuWfPnj0bO+161ateNfj7QQcdtLETx72XXuvtcBbmHXDAAZPPr3nNa2bq59WvfvWmv4899tipgjV0HXjggWun0yLjXYoAJAMWJdKik5mHAQcffPCOYe4ORq3Vd5KCJLH7hGtWodsfryUL0/wdAsuLMAxheN3rXjcTnO+t9n/dJmT05cDmJeyYiQH/b3jDGxZCjmUj0X4M+f0IMGbQ9Z5DDjlkLUSsTl3fNXY8fcLw2te+dn9Di9U1fthhh0110FpCVyEfpi6qva3nxzqJ28XINQnjeib3pje9adQ9tLsoqlS0GBNCGqUMCe28tJlGq231D4ZgvzUwNSmfa2lzX7tjtc5+hjQ/Gb2mmHlf9A3Ga1PVrDqZFBh+63P8KhoM3du6MnKojM/fphF7URMwJHQ71utftg/QR+QkzpADtwzInYYiOZYqpLNq51hndK9MBaOFY7RirEM2Fu7WAYtjfJD9Mip44xvfuIXp0+xpddJWFQoqGDpxjnMW6FZjHSPCC0FZG5hnjWFZl+Pi/20WsjbD+U4GzCKB3AtRfbYFmWMRZBatbY3x6KOPnokY07KSKSx1DnvxCuVsTk8l9CITh4hjhKEKKGM45phjBu9f1LdYNO+wygTZtjqBCsCYFPIYh2mMAKmZmKsWgfm9Cia/iwBDJqMmndbpve8Q1Nj7JisiHHHEEd3/b37zm3uTPgoPjD3qqKMGtb36GvNeYxa4xtBq1QLyXzqudnJqZ2rvNGQ49NBDN17/+tePDhv72ksn8h3veMeWApF6v30qEApYRkmtvhVI58j/3osfw8V3XI6/+jf8xneMmXv4zP86iLbNd1y2ZXt+bn2X9+Z3Xd92CtH9kkklw5xAZUR2RhupfbYhAcYKTNr2I488cqK1Q+sBLU/++OOPn3x+z3ve07WFIFamtpghAvBMa+yMkflWRBlKjO0VPkDfgFcZKiEoErlq8uGHH76JsIxPgZORKTAwmjYSpfj79ttv3/jUpz618f3vf3+iUTq0+AknnHDCxoknnrhx8sknbxEy+uH7t7zlLd3f73vf+3oRaBkMX/UC1dJ8gEW87DF5+xobIwRc2PuWGYIJ73rXuzrUOe2007rvYBz/33zzzd3/MBFhftvb3raJ4AnttJECxjMIyAc/+MHJOHgm51DRcdkRwYojitV10JfgOO6443rtMZ8Zj9pf7XNqSTIy2wa6+fzud7+702y19+yzz+6YSdsnnXTSxkUXXdQJC0wFhej3ox/96MZb3/rWCerVccjwc889t2vf7+pY9qLStmFPXe2AODA0r5b5oI3W79WBoj0YBeHSaWkJzhlnnLFxzjnnbGI+jOR/mFWZ05qoQqEgoenVtME0TRL2nX5TQBmzcI+g8Zk2MEP4UIxFZJjVHAxlAxeNKnZEGAixUosN3W644YZNk9cfeO9739uEQO6F2Cm0p5xyyqZ+3v72t2/xXWASjERwhlYe3/nOd25CKczImWeeORGGFFSdQPvTDxFN+nyn7Uo/r0QAEuqmlXABp2hJrdH/wAc+sImpaDnErdqthmc/PPf+97+/V3MSwlu+RwoDaKRg+r2Mx3zwORGGtjEvmJMckzRhDi0TsupV0LUJwFhJ5j40F6LCFCEVLUPDZCBaSvgGU7kHIUIYLr/88o74wirfwywI/NnPfnbSD3+3vOhEiLTXjEVG4Av85Cc/mYwP4SN3cOqpp3ZMVxjRbL4XqTQtiQgpDBl27tCilflsUU0Jj4kYvBcComkQhLCqFVpVUwFjc2WvT+sRNp0+8wFpTvQfGANCBYNznGizbaeTVxFDH4QxOS5+Q3iGooIdmBtYLJQbI9Uy8fTTT58wML1mkzYf/vCHewl01VVXdRom6qhtEBx4buUPKkLpg/C/KCRDsd3M5bLLLmt69fSfyaX0V0SdmjRCMOhH82YoW2k2jzlYIposp/KmMg7HSOJCgE984hNdLC1z0DTvJUkDEVKr+R3i6ZS1EiYKwZe//OWOIOedd95EM0EyPH20PHP8tFcZJQNcPHKM3Hv++edvcRJTsOi3rkwm7PubwteKtKbtllplvmVPyxnKWoB5FnsgMG3kAGCuhAai9bK1z+kEYlfTll9xxRVdWzJ3jM/Rt5bQ8l80GdAAoUoINwGkwA7lTBAOE0fZj8jhnOx7Go3XlEsYt+o0ZLvSb4Dw2lWk/zOf+UxHNKAVWJWgXB//+Mc750qCoXkSWCaggUA8QoAJUaOEcPpCsNRaGODnGkXoCzheCUwbMkUU4F4QhHvQaNrqY4iCzrOgkP5D+jYpFNChr61tWCJeTkNqcIZ/Epz/IQBwjCP5sY99bMuEscEtuJdwCAKMBjFgCJk4vtdmDwltyynTDCR8I3z4K4Z+qe0IFcItsxGOj3zkI5vSxwglzzM2BAFHEzSjHYTMdjP9vOIkzxgh69dwv3PSVTqT2SZDnBgao402CYMmq2kQQ0eLfgyn8Ada4eW3vvWtwXKvvqiA+zIuz/nwPcKBQCGkIgOMRthMCRMiwkTpUR1E+maOF154YXfPJz/5yU5wyW94Two9985SrjbNnK0cASpcDUltCoUEQ+LxopMgQ/aU/tCkzNqR06df7SgCl548v+VKoI4pgpVZPK8PfehDzb5hrm20MoaOGcRK08e4vvGNb2zKDdivi1JGRLRfI4pl2f4+x6/XhM+alpQAiQY0nlKKHUbb0YSMybH5MJLUampo3+DQKOGb/tDIFDAgNoklw2CipiMTNX0X46Fdxmt/5vjdwg5KtCqKYKjPmDTCXGBi8AEYR0UM2uO5iy++eJTyzcqjGXMN88eVrs8zSDN9X/ziFzt/AK8ZhudauUKCrTcnkIKQkUBqKshx1llnTfXoDb8qozIcRZD0H2ASzGlpunbdyyVm2mZuMDE1Ox1Bhb0VSXz729+e3KMSgR76JOtMFP0XTedvgIkkErBix3cwGi3gN2AQhygJwmSBQGAchKhaqvesdiczLDvns0KRHnZ1JKclWRAaxlv3D/gsv2dcr+PH3Iw20HoZh4DwmTnDcO6hbcu8mDP3G1ZiSvaZolDgnaQMuXv+Bg0y1WrGLtf///a3v00VsnSk0FYzeWg0zFdQIPIFF1ywBUmqg2gmkhA1nbOEXcaNkPKZvmpNIdc111yzJfv43e9+d9M6BmsVjOmrX/3qxAFkPAg9Jq3WRmy7ALRWtVpOXiZqgEcIwMTR1q985StbMmFGCdpLHaRXXnllkmo1HQwjTR+TQUwiC9eigGNEM/nMPTk2mC0qYKa8v3rhQLp1h9Xmwvxrr722G1ude5/pRBm+9rWv9RKecSKomtBqv/sc7RUkh8Z7la7pMwgkl5gcDRSigW4cKjQD6WZyrPeTxFGDc+08zYLJk4zL1U4Picil1ZZjBKP6FrO0sXVlMNPG6TO0Qq+60sg9MDr9FQRb/0ZmffrTn94kyNCRcbT2O7SqlVe8qDS7E8jEgUehVVsMw/BsTfjwHdB36aWXbmkjUeKSSy6ZZNBgMs/SJmGV4RyCJRHRHCAU4sMU0aJuFukLNTVLCBYChz2ujiPPuXDzuc99bkvKF2HXCdSHMbTDyYTBCLi2HqHyd/vi+WT4mIhlZSZg3hSkg04J/9nPftYJwn333dcRCy9ebYdpaErVfjWF//Eh0neokIi2wjids5ryhdjuTzSDV+FZ82RRx5CGOS4Yhrmj/epsIohm/AxzEewf/ehHXXLr97///WS8IoNz0mdpwfsaagj+Z1sUgmm7VW

In [13]:
// WIP remove this restoring code
param.nComp = 1
param.paramFree = [ true, true, true ]

[true, true, true]

### Phasor Analysis

In [14]:
// WIP
param.paramMap = null
phasorRslt = op.run("slim.fitPhasor", param, 2)

net.imagej.slim.FitResults@3783b620

## Other settings

### Region of Interest

Sometimes, instead of the whole dataset, only part of the image (e.g. the region near the nucleus) are of our interest. By specifying the `roi` parameter, we neglect unwanted parts outside of it during fitting. This greatly improves the running time on large images.

In [15]:
import net.imglib2.roi.geom.real.OpenWritableBox

min = [ 20, 20 ]
max = [ 100, 100 ]

// define our region of interest, in this case [40, 87] * [40, 87]
roi = new OpenWritableBox([ min[0] - 1, min[1] - 1 ] as double[], [ max[0] + 1, max[1] + 1 ] as double[])

net.imglib2.roi.geom.real.OpenWritableBox@1d29

We start the fitting routine the same way as before but with the `roi` parameter:

In [16]:
// fitMLA with roi
param.paramMap = rldRslt.paramMap
mlaRslt = op.run("slim.fitMLA", param, 2, roi)
nb.display(fcd.tableDisp(mlaRslt.paramMap, 0, 24, 0, 1000, 7, 14.5))

Z min = -811.3688354492188
Z max = 2383.725341796875
A1 min = -2380.6337890625
A1 max = 1083.6900634765625
Tau1 min = -1.48449153674903552E17
Tau1 max = 2.865592559164457E18


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAkc0lEQVR42u3d5Y41x7XGcbNjx8zMzJCYme2YmZmZQcZYMkS288WSvxiuwvIN+MJ29CvpP+qM3uQcHfnVqOf0SK29p3d1wXqetWpV1arqbbbZZpvVcv2/vhYhLARYroUAy7UQYLkWAizXQoDlWgiwXAsBlmshwHItBFiuhQDLtRBguRYCLNdCgOVaCLBcCwGWayHAci0EWK6FAMu1EOD/fP35z39e7bDDDuP7XnvttTrxxBNXu+222+qoo45a7bzzzqvdd999tc8++4z/XUceeeRql112We26667j/vnnnz/SyOOggw4a+ey0006riy66aHX88cevDj300NXJJ5+8uuyyy1bnnXfe6pZbblldd911qzvvvHN1+umnj/KVu+eee64OO+ywkVYZ6nHcccet9t1339Xhhx8+8j344INXZ5111sj3T3/60+rss88edVFfz6iP9NKqu89tt912fEqzvu39tv7abrvtxucee+wxbwIAwidwXP6mv7tXI0877bTVgQceOARIWKeccsr4/Ygjjhj/n3DCCSMtMPbee+8BOqH7DnjCdA+Y0h977LGrk046aQAKzIsvvniUc9ttt6123HHHAeQBBxwwiIIkwFYHoJ5xxhmDEKeeeurqmGOOWauP9PK85JJLRl7KB5ayPOsZ+e6///6r7bfffouyQJz1vyULbfCnfinFrAkwbQRNWf+7htImgiVsQgZmF0HSOiAgh2f222+/8RsiAPacc84ZeR9yyCFrmuN7n/IG8AUXXDAsiPJoKosATPkC/Yorrhh5IiDSKfvSSy8d/3sG+aRFsCwB68GqqMeZZ5456sUyyU89kVM9EGR926fyQNz/9vusCTA1c1kCggF+ZpCQaTghugB39NFHD2EDgoBplwsZEIUmJeRIBAz5+k06GgoQxACavKSjrQAFtDpkDaqb/FgE9ZBGPi7PX3jhhQMcZKq7QBzpXHfdddewKu6pi3J1B3UJ662gqy6l3/uejDaNDxDTmVTg+p+QgQVcWgic/AH38wtoH80lfCQBALA8z2Trz90H3N///vdBKqACoGcJ1HdA0kp5+F19AOQ3z6gLQl5//fUjnfvKZfqBrhx1k5+6yQMpWQR1cSGdNNro9ynhf/zxxzWZaN9URrqH/+QbzI4AsZ0WxmjsJigOF8EQIPMPMP0kp46Aew6gaRdNRRLP/uUvfxlASudTH88C/PWvf12zPIQJAABn2pUN4HwLZttvCKIsQHsOoMBniVwshu5GHRFHXQJdOlqMjO7L333tQ1IE0k6EkH+KkC9QV5GPsOlGAVOnJ1CAS9sJjIBpHPNKkO7TIgK/+uqrx/+ETjulBz5tRCigeV6a22+/fTyva5C2kUKjCdqmHM/5vYu59jwg+Qryf+SRRwZowFEmP4K5V5b7CAFg9/Md3A9EZSl36rgqC4EQ1v0tgY28nvPbBvgBWy9z2qbR+seGdgRCaAQMLNrP8UqQhHvuuecOkNM6/wNYXlddddUQpufkB0RA5HxOBcwkc/rK2//SMunyd798gEdrlfm3v/1tWBdWyfO0nJWJlMCVR46s55Wb8+p/7fCc73VFQP5Pnv6W/ITZEgDYzDHQ6s+BDcy8e4KU1vebb755pNGnExLBSteIAJGQxP08f0Kl2YDQDdBgpj0gp/5HQlcH3QCtB5DfEcJziEBjr7322lGe8iMd83733Xevle/TfXl4PtLJW309p8tqBOHyv3rI2ydN12WQk89N1QU05q5fdNEkprY+Vr8ONODT8BtvvHF0B0wuwUmHFH6jie4TeASSNyCQ5P777x+fyqClvgMVWPIgbOC6EM/zzUX0nHsRtnp6niVAFPfUW11YAGa9y/+AT9PlqaypU9vE0nRyKODVr25g9gTIYWpGrvE4gdJWWgsUGgF8AsrrJhxCQR6aBWRj9yeffHL4EZ6XHhAJk/CUKb37rANiELr7gIkovtPWNF8aQClfueqaY6eLYN7VBZn4H+773bOekwaRqk/zFIiNsJ5RLlJmAaYTRFvq77c0PzArAhAKU04b9Jt5zkw4gdJ8wgEqYqStTCGhAqm5A2BLc8899wxNlFcazdzSGISiwZ4DYmYb0Gk1rWx6OI8cUQCGWADSPTRFLG+k0Rbk4JhWnjoCExE8q/7uI726IixCkAECqIP/1ZF8kMP/5LPeWV4/kTZLAhAUYRI4gRI4AhBI/ToSAMbsGeCAARwEQQ73gEPwBE24hOkTERoCEqL+Pw2dTqRkagk4ZzRT6566KVcXQyMbtgJcPX2Xb+Bl/j2LHIDVjoaKzTVok1GMi1+BLOqq7jmR7iGX8jdo9m/rEQDQwGscTNsIkDCbaSPkxvWEDyAX8JBAGpp25ZVXrpnoF154YYBMYE3uBFrA5W/k8buaBwAScimvMbj76oRw+QHqoc5IO7UEgC6tOl5zzTVrI5vWEpBRGmsP5KD+2qKeniEXpKhLahp7g6eD/9gMCajpVownAMIlJBrU5IzfkWCqrbQTuLSMaQcax5DQmuYFEpPrOx+BhWjMzeqwLA3Jmg4GJjI2XFMWEJQvPeBopro1i9iETnMLtU8Z/levyy+/fDyDfNpG6+Wpzf7Xfu2QFwXQPvU2LEYYz27gDODWIYCGaXTjd6adYIB5ww03jE/sJ0SgtrLnE2AIQIBAkBdBykueAR1YiOR7TpVPeSkT6XLsEIZmIgwAEK6JnFYdfTcEBDiS+F89kNEnENUHYXzKX37A1PerO1N/0003DW1vBCNvBOFb+KyLUW++zQb2/VuHAIRI02im74TC1DOXgNFwoOhTCdongAilaVPPMqHAyhOncUBrurVVOyBEqIZ3rc8Xd9A8RDOL6qWsVhjlrw6AzgSra12a9jRNrE7Ad1/bmr9AbHVRBudXmiyO7kIZiGIiy2eTVtofgddPEc+SAHnpQCQ0ggTeo48+OgSi8bSi6dQE2/IpgQCXtQAgcJ9++um1fhqZCK5lZRosL8Dnb9DSxuhABhBwAdQoAyAI47cWiVrvb35A/Rql0OCmsXVn+THFCzQ/wTFs6Os53QSZSOM3slAX5apvM5nTOZRZE8B8OpNPozhDBNHQLEcI6EwloXz99dfjPtAJ1W+ERlCET1CtGEpHyEBq7h8JPCctQET+eMb3gKORwFc/JHQ/J7U1ffVN+E1cZbGU0dqF9I1OrEOoV/6EskxoedY9zz/wwAPDeW1VkRzIB2Ea+WwA6FuPAPq9AAA+oAk08Gig3wiSln/55ZejfyYMZrpJIc+9+uqrIz/PegYY8mYBaFH9ZmP5fApCRhLPIIb8ix5SRnEFQARaq4NN1SIJwshLWn3+fffdNwgN3HwQpEDYZhVbA6gL0cezWC106Qoy+cptWrlYCX7EtDuYJQFohzV1ACIAi3DHHXeMCRIOk6GRNITIuycYwtIvA5p2tAoHUAJnNYBXxFBaKg3h1+83fGyauCGh53wvbIy291sTO0UE5Tj6DnykyGpUZs4jMiOR51vJRGwxiC1V+03bkIAMlNfIw2dL1E1QzT4mUCNfe+21wXSNNE9P04FNm4GpkUwlgjDPSIEEReMQKjLwG2isZxpDy7dIobzvooXqv93zHMsgv2bdaBch00rE8n/rBZ73XNO9AI8AIn4aPhZEKg9tU6/pHIY6IH9kUM7rr78+2k4OTz311NpaRtahYNC6kVkTQD/57LPPjr6Y8/bggw8OQWisT/27TxG6tEQfSVhpEEHRIOA2T8AfkDZnq4UhIBXZy8y3np4Jb2WQ1tHeVhDVrfp6Vp66lpZ6kaeAEb/deuutQ4uRFakLZ6uvV2b1V28+i7xZPhbQmofuTLsbDdW11DUpu+niWRMAWLTb8imhEgIhEhjLoB+kyffee+/4HTF41rqN5tYJEWjN2hG+/tRz8pG

In the results above, all other regions outside the box is neglected.

### Binning

Binning settings are enabled by setting the binning kernel parameter. The kernel can be any image. Here we use the built-in `SQUARE_KERNEL_3`, a $3\times3$ image with each pixel valued $\frac{1}{9}$:

In [17]:
import net.imagej.slim.SlimOps
SlimOps.SQUARE_KERNEL_3

In [18]:
import net.imagej.slim.SlimOps

// spin!
rldRslt = ij.op().run("slim.fitRLD", param, lifetimeAxis, roi, SlimOps.SQUARE_KERNEL_3)

param.paramMap = rldRslt.paramMap
mlaRslt = ij.op().run("slim.fitMLA", param, lifetimeAxis, roi, SlimOps.SQUARE_KERNEL_3)

// z: [0, zMax], a: [0, 900], tau: [7, 14.5]
nb.display(fcd.tableDisp(mlaRslt.paramMap, 0, null, 0, 900, 7, 14.5))

Z min = -Infinity
Z max = 10.23491382598877
A1 min = 0.0
A1 max = Infinity
Tau1 min = 0.0
Tau1 max = 2.097152E8


Z A1 Tau1 Pseudocolor1 <img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAYAAADDPmHLAAAni0lEQVR42u3d9882VbXGcQQFxAKCHQUFEXsvIPZCFRULVuwFKwhYEUFRUUHAAopgpQkq9g5YwN6xC5poIMbEH/wPxnx28n1zn/dw/OHkxSfzOk+yM/dz3zN7ZtZ1rWutXWbPJptsssm0lP/qshhhIcBSFgIsZSHAUhYCLGUhwFIWAixlIcBSFgIsZSHAUhYCLGUhwFIWAixlIcBSFgIsZSHAUhYCLGUhwFIWAixlIcD/q9zgBjcY5cY3vvG0ww47TPe9732ne93rXtPuu+8+3fnOd57ufve7T3e4wx3G9w984AOnRz3qUdNuu+023eUud5l22WWX8b197Xfb2952uvWtbz3KPe95z7HvE5/4xOmxj33s9JznPGd66lOfOh1++OHT85///OnII4+cnve8503Pfe5zp3322Wd6wAMeMO2xxx7TfvvtNz396U+fHv3oR4/jH/KQh4z673GPe0x3vetdx+c999xzuv/97z/d5z73mXbaaadxLv9Xh+t1Dde73vWmrbbaatpiiy3W3e+mm246bbbZZtO1rnWtsf2vJgAD3eIWtxjAMfD97ne/YVxAP/ShDx0GBjZwGdbvD3rQg8b3PiPKXnvtNYBifGDc8pa3nLbffvvpTne60/SIRzxietzjHjdI8MIXvnCA/frXv356wQteMB1zzDHTi1/84lEOOOCAUez3lKc8ZXrCE54wCPCkJz1pesxjHjMKgJHNdey///7TvvvuO0qkRCJl7733Htfimm9zm9tMW2+99XTd6173f5BglQzKda5znbFFivX3cexGR4DNN998uuENbzhtt912081udrPh+Q9+8IMHCM9+9rOHx2ZIgNs+7GEPG8Ar/Q8MAADad37bddddR6EOyPPwhz98FIA+8pGPnB7/+MeP/QENwIB/2tOeNj35yU8e+wWmfSkCAqjDtSAmwCOerd+RBtmoDAVBHqRxb+7zRje60f8CM9B9TylWf0eKLbfccthqoyIA2WOMm970psNbecnd7na3AcpBBx00vBM4AOZxQPSZ4X3m2UCnFIzvN2DxSsV+lOF2t7vdqDfSAEOxPyADFtEU5FAniW+/Aw88cADqehzTPggaSf2uHgRSJ+CR2P+uDxEpHKJf//rXv1pAfUchVhUgdVA2GgKI9ZhO9sm1WCnOMzgQGVNcfsYznjGMC7h73/vew/AIwvi2jMxbHeN3xODNBx988JB6n8Vrhlf/He94x7GVNyBF+YT47ligq8f3pFtMpygBy7sRAsGoguukHD4rzgd81+zaI4D6nde1IC5S3upWtxqKwLtXbYMc64eAqwsJsyUAhpM1hQGAw+gAZWDexmjPetazRqxmYB5EkhncPowMBErx0pe+dHim44EICAmeGA8EOQJjM7oCAISzlW8A3XE8GwERAsgBScZf8pKXTC9/+cvHeYDpHGReQhkB7IuYxX+fhTNkQiQKYKsgtGspLMgPrn3ta69zjo02CSRxGC7pkx2L0XkugwMM+Az9zGc+czrkkEMGADwL4L5TxGoK8aIXvWiA439xWHy27yte8YpRkImnCzM3v/nNh/wiAzCcV4hYbUUggfNQlSTdtbguqoKQSOcaSH/Xiqz2dQ/IihS1UlZVxzkoivDk/H5bXwnWINb/5wiA6YC4yU1usk6GazJlbN4T2K95zWuGh5N6xXcAYOBkGUEAXZKn2A8ofuOxpPf2t7/9IJ2wo3UAFATceeedhyfyfl7Ls3m086kf6EimySislAsIR5GBEkRA4NY8VL+63SuvRzCkiBhUYMcddxxF8xcRNupm4DbbbDNulmFq2gG7phbAeBajMvpb3vKWQQJG59n24+3+Bw4ASsTINu8j6cAocUMWJON9iFAY4P1CQc1PgAkjQEJI8q0+BKofAQGATRH85rNy6KGHDtLk9fVhlHeUhPabc8kx6rOgAlSRfVaTPbF/22233TgIQPp5W213sZbHAEkBJs9CAt5G/g877LAh5YwrtvKsyKKVgCQ8nfTySHWXbNW0U7ffAMvgyTHwkaG+B8QBvN+AJlYjlWNr6yMBhUipqAMiUinX5xzqUrd7FFpcizrdq7rsh2gKtShJZR/xn0qu30yuw2jWBGAMXsUAjMlrgaMDhgeJsa9+9aunN73pTdPb3/72AT5D24/hGAtIjIwAxx577PTWt751eu1rX7sOGN7K2EDO2EIFBeB1gVGWX/E7wPwmVCBGUo2oQK2ZSVmQDmHlIUjmO4Dap9geEV0T4sorajlIZN0DOyCafTWFayaunwjWSTRrAjBMbXaFIXjxy172spHM8XqZ9hve8Ibpla985boOFAXwgPAZARjwqKOOGiHida973ZBhIAAbuIgGVMpgy8jiP6lV6ipGSvLsd2HB92I2AgBPSTHqleT5ziMMCFWu0/0gCxIhQZ1SrsU9IwlFQ3A9j5RLQYr6LByrDuERCXj/+v0BsyYA0HgzL2Ik3kA+kaBYSg2OOOKI8ZlR6h9gUAUReDajCw1U4B3veMcgjboBIQw4lvEZFwGQrw4nWyRAAAAzvH2BXo+kTqlaJwjSWIR9kcv1U5wyf/s5Ty0AJCkZpBQIjrBHH330UDdETz1qdahLna5Jknp1XcazJwCjMCDjFqNrS5NxvwMSIAzPG3lpHUFiNOLoYOFBr3rVq0aSyKA8jtzWp9AgkGMlZIAFOpAZWDKqPsfYx+8SMa0Ux7qWksrGG+od9P9qp5Fr9T0yCA/UDUndF/DlClTN9b75zW8e5HUPQEcA9+y+hDz2cc/rh4E1GDDasBUyBIYjAhXgAcAni37jKfXKkVweBzBem2wzfKSRIzhWDkCKHVvvHTB4LVnnzWTVZ4DVCSRO89CSM0anAnIAxODFgAGkz+ovg48QjrGtl7L+f8pWK4T0y3He+MY3DqLqpJLv2Mf5hTTOwC41aZELQTcqBXDTNeF4BNlmHF4h/rt5hmdQACW9gGIMxmYYhkIYoULxmQFrugEUYDW1tDyK6YBSJxL4zf4IR9Jr9jne/+oEEM/0P4mv44jkI5r66j10TlKOnFoFrosauDdk7zv3ztMbbQS4Yj9OwD7uxXWudgrNXgG0oSVObpIEkkIZv9jImxmOJKYCNcEYHRkYHYEYVT28CQF4lO8BUPPK8YDhzRTAlsw3t6CewNr8qUBteXUhAwIo9tESqIlIAdTvPK7Z+QELaNLuHt1PgPqOAwgD9WYivH2EMvbgDParyYtw1G8Nkr9rhgDkuoSPIWTDbhygSWaf7ctjEIMxtP397jgGQwAJoxYAJeFN9hO3yzMAXrMPULW/kYkiAL/RRWqAYJ2rIV1EALxcRPJYj17kEL+FpMYLumbXBvC6sV27a0RaKoEsPrOB0CCPEcraHxndA7I1OrgGSeGGrZBhkkPA89wMUo8ez/Db2WefPX3oQx+aTjnllOFhjCwWU4u3ve1tI5tW1KFehEg9AFwvnK24re4mbzScHGF8ll9QCrLO8PZvX/lDrQehqZYAErgmsu4axHnkdn+AbCg7wqiz7mnA68PQjDVo5Vj1uE/7IaFzOJ/ElAqsP2o4OwIcd9xxQ/JJv6bbO9/5znHzjMYjGIdCyAk+97nPTZ/4xCdGqVfQfgDnKQzIaxgbgWTfPJqs56XiPEDJPsPah9cCpc4fZFFKMuu2rcMJOdSDABJSpKpV0LQx1y28US6koRyIweuboQRQ+1EJZEFeNqgzi1pwAopT81L9yKcn0AQRPYWzJsB73vOecdPvete7xucPfvCDo5iT11ArI5D5M844YzrttNOmd7/73YMASvP5SGZjBKvNqDpeGm7l/UIAVQDC6ryA1KEZQ5p+wK8zpzrqPTRnwTGNIKpndZ6AbTODhA+kRLpaDX5zD9QN4O7h5JNPHg7x/ve/f6gacjuW/Ks